# AutoPrognosis API Tutorial

A demonstration for AP functionality and operation

This tutorial shows how to use [Autoprognosis](https://arxiv.org/abs/1802.07207). We are using the UCI Spam dataset.

See [installation instructions](../../doc/install.md) to install the dependencies.

Load dataset and show the first five samples:

In [207]:
import pandas as pd
import numpy as np
import initpath_ap
initpath_ap.init_sys_path()
import utilmlab
import json;
from scipy import stats
from sklearn.datasets import load_breast_cancer

#df = load_breast_cancer()
#X_ = pd.DataFrame(df.data)
#Y_ = pd.DataFrame(df.target)

## Import the AutoPrognosis library

In [28]:
import model

# Run the model from command line

## introduce with a sample

--it : total number of iterations for each fold or n-fold cross validation

--cv : If 0, that means a normal validation with train and test (or validation) set. -iValIndex should also be set. Otherwise, n for n-fold cross validation

-iValIndex: address of the test index file, test_indexes.csv or val_indexes.csv", 

--nstage: size of pipeline: 0: auto (selects imputation when missing data is detected),
        1: only classifiers, 
        2: feature processesing + clf, 
        3: imputers + feature processors and clf
        
--ensemble : include ensembles when fitting. It gives an assertion error when set to 0! should be looked into.

--modelindexes : list of 

0 Random Forest,
1 Gradient Boosting, 
2 XGBoost, 
3 Adaboost, 
4 Bagging, 
5 Bernoulli Naive Bayes, 
6 Gauss Naive Bayes, 
7 Multinomial Naive Bayes, 
8 Logistic Regression, 
9 Perceptron, 
10 Decision Trees, 
11 QDA, 
12 LDA, 
13 KNN, 
14 Linear SVM, 
15 Neural Network

In [ ]:
!python3 autoprognosis.py\
-i ../../../AutoPrognosisThings/cardio_data/withImage_cardio_data_17_feature_noNan.csv\
--target outcome \
-o ../../../AutoPrognosisThings/outputs \
--it 15 \
--cv 3 \
--nstage 4 \
--modelindexes 2, 3, 4\
--num_components 3\
--kernel_freq 100

In [ ]:
!python3 autoprognosis_report.py -i ../../../AutoPrognosisThings/outputs --verbose 1

score_d or result.json contains the initial 4 scores: classifier/ensemble aucroc/aucprc, and is the score and other information of the AP_mdl that is passed to evaluate_ens(). This function makes a deep copy of this model and returns that. In autoprognosis.py, where this function is called, the report_d is then produced out of the deep copy that is passed. I wonder why it is so complicated and why did the model deep copy itself.

report_d or report.json contains the numbers in the middle: after the word Report, and before the long list. This was produced by APReport in model.

clf_d or result_clf.json contains the long list of all pipelines


holy mother of God! From the code, it looks like that the cross validation is done first, and for each train and test set, the model is updated for n (number of iterations) times, instead of fixing a model and testing it on train and test set splits.

## Run for image data set with different categories

### no image information yet

In [192]:
!rm ../../../AutoPrognosisThings/outputs/agg_green.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage_green.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2\
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_green.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_green.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

rm: ../../../AutoPrognosisThings/outputs/agg_green.txt: No such file or directory
rm: ../../../AutoPrognosisThings/outputs/result.json: No such file or directory
[ XGBoost ]
[ XGBoost ]
Iteration number: 1 1s (1s) (148s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 2s (1s) (102s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0000000000000038
[ XGBoost ]
Iteration number: 3 3s (1s) (85s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.385970518045935
[ XGBoost ]
Iteration number: 4 3s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.368075094303016
[ XGBoost ]
Iteration number: 5 4s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2272278703108626
[ XGBoost ]
Iteration number: 6 5s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.23378771475434
[ XGBoost ]
Iteration number: 7 6s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2765861330721493
[ XGBoost ]
Iterat

[ XGBoost ]
Iteration number: 68 51s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4144656216099967
[ XGBoost ]
Iteration number: 69 52s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4018705161771303
[ XGBoost ]
Iteration number: 70 52s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4164956288320028
[ XGBoost ]
Iteration number: 71 53s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4263701993216322
[ XGBoost ]
Iteration number: 72 54s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4412736701528015
[ XGBoost ]
Iteration number: 73 55s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4531739822295273
[ XGBoost ]
Iteration number: 74 55s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4464082673981413
[ XGBoost ]
Iteration number: 75 56s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4436990611110714
[ XGBoost ]
Iteration number: 76 57s (1s

[ XGBoost ]
Iteration number: 25 21s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4704697256530381
[ XGBoost ]
Iteration number: 26 22s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4889002689746558
[ XGBoost ]
Iteration number: 27 22s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4574874972200316
[ XGBoost ]
Iteration number: 28 23s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.449497344439915
[ XGBoost ]
Iteration number: 29 24s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4551420886306414
[ XGBoost ]
Iteration number: 30 24s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4707403884641963
[ XGBoost ]
Iteration number: 31 25s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4387194088746669
[ XGBoost ]
Iteration number: 32 25s (1s) (79s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4651487871681728
[ XGBoost ]
Iteration number: 33 26s (1s)

[ XGBoost ]
Iteration number: 94 66s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4649833945341233
[ XGBoost ]
Iteration number: 95 67s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4690250322443956
[ XGBoost ]
Iteration number: 96 68s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4629682578011969
[ XGBoost ]
Iteration number: 97 68s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.465368254540868
[ XGBoost ]
Iteration number: 98 69s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4740092667660816
[ XGBoost ]
Iteration number: 99 70s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4601135151912905
[ XGBoost ]
Iteration number: 100 71s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.444037587861467

{'model_list': [<models.classifiers.XGboost object at 0x1a20aae750>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boo

[ XGBoost ]
Iteration number: 51 40s (1s) (78s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2845312508739954
[ XGBoost ]
Iteration number: 52 40s (1s) (78s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2977021976522973
[ XGBoost ]
Iteration number: 53 41s (1s) (78s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3056582451761154
[ XGBoost ]
Iteration number: 54 42s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3138042037793976
[ XGBoost ]
Iteration number: 55 42s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3005274151039856
[ XGBoost ]
Iteration number: 56 43s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.300669236035971
[ XGBoost ]
Iteration number: 57 44s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3169705297502252
[ XGBoost ]
Iteration number: 58 45s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3263370475393779
[ XGBoost ]
Iteration number: 59 45s (1s)

...

[ XGBoost ]
Iteration number: 52 40s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2099479834573135
[ XGBoost ]
Iteration number: 53 41s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2179836161990885
[ XGBoost ]
Iteration number: 54 42s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.230519007566561
[ XGBoost ]
Iteration number: 55 42s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2238131324588137
[ XGBoost ]
Iteration number: 56 43s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.221363639081959
[ XGBoost ]
Iteration number: 57 44s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2327670969058422
[ XGBoost ]
Iteration number: 58 44s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2231083060367427
[ XGBoost ]
Iteration number: 59 45s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.228967010278269
[ XGBoost ]
Iteration number: 60 46s (1s) (

Iteration number: 8 7s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.38142620565198
[ XGBoost ]
Iteration number: 9 8s (1s) (87s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.400911512693862
[ XGBoost ]
Iteration number: 10 9s (1s) (85s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5012398324848983
[ XGBoost ]
Iteration number: 11 9s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4588329254924437
[ XGBoost ]
Iteration number: 12 10s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3888251197763044
[ XGBoost ]
Iteration number: 13 11s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2609725342365776
[ XGBoost ]
Iteration number: 14 12s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.317862897791194
[ XGBoost ]
Iteration number: 15 13s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3049078116651072
[ XGBoost ]
Iteration number: 16 13s (1s) (84s), Current pipel

[ XGBoost ]
Iteration number: 77 69s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4308026767731812
[ XGBoost ]
Iteration number: 78 70s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4120092852684678
[ XGBoost ]
Iteration number: 79 71s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4208886669232454
[ XGBoost ]
Iteration number: 80 72s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4044345542555734
[ XGBoost ]
Iteration number: 81 73s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4131082328727307
[ XGBoost ]
Iteration number: 82 74s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4258820079581591
[ XGBoost ]
Iteration number: 83 75s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5641344769353616
[ XGBoost ]
Iteration number: 84 76s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.576262424532115
[ XGBoost ]
Iteration number: 85 77s (1s)

Iteration number: 33 39s (1s) (118s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2200925615539167
[ XGBoost ]
Iteration number: 34 40s (1s) (117s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2322008264761968
[ XGBoost ]
Iteration number: 35 41s (1s) (118s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3884461735110036
[ XGBoost ]
Iteration number: 36 42s (1s) (118s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.403850101386975
[ XGBoost ]
Iteration number: 37 44s (1s) (119s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4242456791899107
[ XGBoost ]
Iteration number: 38 44s (1s) (117s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4452150797105738
[ XGBoost ]
Iteration number: 39 46s (1s) (118s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.431400198791255
[ XGBoost ]
Iteration number: 40 47s (1s) (117s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4524635727031592
[ XGBoost ]
Iteration number: 41 48s (1s) (117

{'model_list': [<models.classifiers.XGboost object at 0x1a26538710>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes': None, 'num_stages': 1, 'pipeline_stages': ['classifier'], 'name': '[ XGBoost ]', 'analysis_mode': None, 'analysis_type': None}
{'model_list': [<models.classifiers.XGboost object at 0x1a265387d0>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes': None, 'num_stages': 1, 'pipeline_stages': ['classifier'], 'name': '[ XGBoost ]', 'analysis_mode': None, 'analysis_type': None}
{'model_list': [<models.classifiers.XGboost object at 0x1a265388d0>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes':

In [193]:
!rm ../../../AutoPrognosisThings/outputs/agg_orange.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage_orange.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2\
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_orange.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_orange.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

rm: ../../../AutoPrognosisThings/outputs/agg_orange.txt: No such file or directory
[ XGBoost ]
[ XGBoost ]
Iteration number: 1 2s (2s) (200s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 3s (1s) (129s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0000000000000189
[ XGBoost ]
Iteration number: 3 3s (1s) (105s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7214860783032891
[ XGBoost ]
Iteration number: 4 4s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7498580864766771
[ XGBoost ]
Iteration number: 5 5s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1224061817838493
[ XGBoost ]
Iteration number: 6 7s (1s) (121s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.129489845444874
[ XGBoost ]
Iteration number: 7 8s (1s) (112s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2766950039661684
[ XGBoost ]
Iteration number: 8 9s (1s) (112s), Current pipelines:  [[[ XGBoost ]]], BO obj

[ XGBoost ]
Iteration number: 68 100s (1s) (147s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9295345533769782
[ XGBoost ]
Iteration number: 69 100s (1s) (145s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9185172579206079
[ XGBoost ]
Iteration number: 70 102s (1s) (145s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9079200301848044
[ XGBoost ]
Iteration number: 71 102s (1s) (144s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9143027833119269
[ XGBoost ]
Iteration number: 72 103s (1s) (143s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9041068580778152
[ XGBoost ]
Iteration number: 73 104s (1s) (142s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8942787875275408
[ XGBoost ]
Iteration number: 74 104s (1s) (141s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8880033074027689
[ XGBoost ]
Iteration number: 75 105s (1s) (140s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8787841719545363
[ XGBoost ]
Iteration nu

[ XGBoost ]
Iteration number: 24 28s (1s) (117s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9655371753030617
[ XGBoost ]
Iteration number: 25 29s (1s) (116s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9587110126941878
[ XGBoost ]
Iteration number: 26 30s (1s) (114s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9974347827177521
[ XGBoost ]
Iteration number: 27 31s (1s) (113s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0125947877475836
[ XGBoost ]
Iteration number: 28 31s (1s) (111s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0253594862012576
[ XGBoost ]
Iteration number: 29 32s (1s) (110s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0444273199518568
[ XGBoost ]
Iteration number: 30 32s (1s) (108s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0218341222573972
[ XGBoost ]
Iteration number: 31 33s (1s) (107s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0027404227556784
[ XGBoost ]
Iteration number: 32

[ XGBoost ]
Iteration number: 93 90s (1s) (97s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0642137946206502
[ XGBoost ]
Iteration number: 94 91s (1s) (97s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0704669898785975
[ XGBoost ]
Iteration number: 95 92s (1s) (97s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0751186651119105
[ XGBoost ]
Iteration number: 96 94s (1s) (97s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.104960305867683
[ XGBoost ]
Iteration number: 97 96s (1s) (99s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1058079705552308
[ XGBoost ]
Iteration number: 98 98s (1s) (100s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0998178014965925
[ XGBoost ]
Iteration number: 99 100s (1s) (101s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.103874510756315
[ XGBoost ]
Iteration number: 100 104s (1s) (104s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1060295961302113

{'model_list': [<models.classifiers

[ XGBoost ]
Iteration number: 49 66s (1s) (135s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9357592198316632
[ XGBoost ]
Iteration number: 50 68s (1s) (136s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9385686230716499
[ XGBoost ]
Iteration number: 51 69s (1s) (135s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9435213519672171
[ XGBoost ]
Iteration number: 52 70s (1s) (136s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.937780759600541
[ XGBoost ]
Iteration number: 53 71s (1s) (134s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9435945837493627
[ XGBoost ]
Iteration number: 54 72s (1s) (133s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9350262227083427
[ XGBoost ]
Iteration number: 55 72s (1s) (131s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9273375240596308
[ XGBoost ]
Iteration number: 56 73s (1s) (130s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9427446337357193
[ XGBoost ]
Iteration number: 57 

...

[ XGBoost ]
Iteration number: 37 27s (1s) (72s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4257611632676392
[ XGBoost ]
Iteration number: 38 27s (1s) (72s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4320126795786954
[ XGBoost ]
Iteration number: 39 28s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4224485159886808
[ XGBoost ]
Iteration number: 40 29s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4404713543178365
[ XGBoost ]
Iteration number: 41 29s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4145731723077077
[ XGBoost ]
Iteration number: 42 30s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4390424091965406
[ XGBoost ]
Iteration number: 43 30s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.419117133642836
[ XGBoost ]
Iteration number: 44 31s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4430602889416262
[ XGBoost ]
Iteration number: 45 32s (1s)

[ XGBoost ]
[ XGBoost ]
Iteration number: 1 2s (2s) (163s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 3s (1s) (131s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 4s (1s) (144s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.298357644138611
[ XGBoost ]
Iteration number: 4 6s (2s) (158s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0337617183911623
[ XGBoost ]
Iteration number: 5 8s (2s) (157s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0354741475214928
[ XGBoost ]
Iteration number: 6 9s (2s) (156s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8766362830394402
[ XGBoost ]
Iteration number: 7 11s (2s) (159s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9999093549909416
[ XGBoost ]
Iteration number: 8 13s (2s) (158s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9924847087010638
[ XGBoost ]
Iteration number: 9 14s (2s) (154s), Current pipelines

[ XGBoost ]
Iteration number: 68 95s (1s) (139s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9919772169257522
[ XGBoost ]
Iteration number: 69 96s (1s) (139s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9912404901621323
[ XGBoost ]
Iteration number: 70 97s (1s) (139s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9915477813712119
[ XGBoost ]
Iteration number: 71 98s (1s) (138s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.995939657231837
[ XGBoost ]
Iteration number: 72 99s (1s) (138s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.004439698603907
[ XGBoost ]
Iteration number: 73 100s (1s) (137s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0181417883085477
[ XGBoost ]
Iteration number: 74 101s (1s) (137s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0317500397126262
[ XGBoost ]
Iteration number: 75 102s (1s) (136s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0399970416207818
[ XGBoost ]
Iteration number: 7

[ XGBoost ]
Iteration number: 24 20s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3788379417359309
[ XGBoost ]
Iteration number: 25 21s (1s) (85s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3457560010018295
[ XGBoost ]
Iteration number: 26 22s (1s) (85s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2488945318978288
[ XGBoost ]
Iteration number: 27 23s (1s) (85s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2646719423456096
[ XGBoost ]
Iteration number: 28 24s (1s) (86s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2635698491905278
[ XGBoost ]
Iteration number: 29 25s (1s) (86s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2718245730469517
[ XGBoost ]
Iteration number: 30 26s (1s) (85s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3059685977247073
[ XGBoost ]
Iteration number: 31 26s (1s) (85s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3200940138074104
[ XGBoost ]
Iteration number: 32 27s (1s

[ XGBoost ]
Iteration number: 93 74s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1693588739383276
[ XGBoost ]
Iteration number: 94 75s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.173251635204832
[ XGBoost ]
Iteration number: 95 76s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1768889672319331
[ XGBoost ]
Iteration number: 96 77s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1786817576296509
[ XGBoost ]
Iteration number: 97 78s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1809602973207711
[ XGBoost ]
Iteration number: 98 78s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1854160983311028
[ XGBoost ]
Iteration number: 99 79s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1848541217475672
[ XGBoost ]
Iteration number: 100 80s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1870336202295468

{'model_list': [<models.classifiers.XGb

In [194]:
!rm ../../../AutoPrognosisThings/outputs/agg_Vascular.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage_Vascular.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2\
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_Vascular.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_Vascular.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

rm: ../../../AutoPrognosisThings/outputs/agg_Vascular.txt: No such file or directory
[ XGBoost ]
[ XGBoost ]
Iteration number: 1 0s (0s) (44s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 1s (0s) (48s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0000000000000164
[ XGBoost ]
Iteration number: 3 1s (0s) (49s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9303578865305514
[ XGBoost ]
Iteration number: 4 3s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9708211888144508
[ XGBoost ]
Iteration number: 5 4s (1s) (76s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8340779165484483
[ XGBoost ]
Iteration number: 6 4s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9654200855155338
[ XGBoost ]
Iteration number: 7 5s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0073694253771452
[ XGBoost ]
Iteration number: 8 6s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO object

[ XGBoost ]
Iteration number: 68 43s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2582553846811408
[ XGBoost ]
Iteration number: 69 43s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2491656662082662
[ XGBoost ]
Iteration number: 70 44s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2551625239930138
[ XGBoost ]
Iteration number: 71 45s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2399822284668047
[ XGBoost ]
Iteration number: 72 45s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2248362047735335
[ XGBoost ]
Iteration number: 73 46s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2082254219561424
[ XGBoost ]
Iteration number: 74 47s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1980140585960228
[ XGBoost ]
Iteration number: 75 47s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1811403816681332
[ XGBoost ]
Iteration number: 76 48s (1s

[ XGBoost ]
Iteration number: 25 22s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9988253815091303
[ XGBoost ]
Iteration number: 26 22s (1s) (86s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0196460307108792
[ XGBoost ]
Iteration number: 27 23s (1s) (85s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0408932526742753
[ XGBoost ]
Iteration number: 28 24s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0376468798438978
[ XGBoost ]
Iteration number: 29 24s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0253311684164885
[ XGBoost ]
Iteration number: 30 25s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0345459019442733
[ XGBoost ]
Iteration number: 31 25s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0528536888845328
[ XGBoost ]
Iteration number: 32 26s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0432806342127183
[ XGBoost ]
Iteration number: 33 26s (1s

[ XGBoost ]
Iteration number: 94 97s (1s) (104s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.329830905209509
[ XGBoost ]
Iteration number: 95 98s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3366594561299685
[ XGBoost ]
Iteration number: 96 101s (1s) (105s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3458866675871022
[ XGBoost ]
Iteration number: 97 102s (1s) (106s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3526923347582267
[ XGBoost ]
Iteration number: 98 103s (1s) (105s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.357470406944386
[ XGBoost ]
Iteration number: 99 105s (1s) (106s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3554989600105218
[ XGBoost ]
Iteration number: 100 106s (1s) (106s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3654030386966145

{'model_list': [<models.classifiers.XGboost object at 0x1a1cb34690>], 'explained': '[ *GBoost is an open-source software library which provides the 

[ XGBoost ]
Iteration number: 50 51s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2722072633071095
[ XGBoost ]
Iteration number: 51 52s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2401482205413308
[ XGBoost ]
Iteration number: 52 54s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2269481424258717
[ XGBoost ]
Iteration number: 53 55s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2282251716989743
[ XGBoost ]
Iteration number: 54 56s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2123809423203542
[ XGBoost ]
Iteration number: 55 56s (1s) (102s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2202228097425085
[ XGBoost ]
Iteration number: 56 57s (1s) (102s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.205018818854455
[ XGBoost ]
Iteration number: 57 59s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1826849004016824
[ XGBoost ]
Iteration number: 58 

...

[ XGBoost ]
Iteration number: 41 28s (1s) (68s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.142515330262543
[ XGBoost ]
Iteration number: 42 28s (1s) (68s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1608467676488492
[ XGBoost ]
Iteration number: 43 29s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1650205837869514
[ XGBoost ]
Iteration number: 44 30s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.145445132494899
[ XGBoost ]
Iteration number: 45 30s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1482234309642243
[ XGBoost ]
Iteration number: 46 31s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1507306683376737
[ XGBoost ]
Iteration number: 47 31s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.155751146620516
[ XGBoost ]
Iteration number: 48 32s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1489555485473202
[ XGBoost ]
Iteration number: 49 32s (1s) (

[ XGBoost ]
[ XGBoost ]
Iteration number: 1 5s (5s) (467s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 5s (3s) (259s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 6s (2s) (191s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3220240075314884
[ XGBoost ]
Iteration number: 4 6s (2s) (156s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.027390911877533
[ XGBoost ]
Iteration number: 5 7s (1s) (138s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.172208907828514
[ XGBoost ]
Iteration number: 6 7s (1s) (125s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1280260751221003
[ XGBoost ]
Iteration number: 7 8s (1s) (117s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0254028724836015
[ XGBoost ]
Iteration number: 8 9s (1s) (111s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.135167088514716
[ XGBoost ]
Iteration number: 9 10s (1s) (108s), Current pipelines:  [

[ XGBoost ]
Iteration number: 69 48s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3682597780349355
[ XGBoost ]
Iteration number: 70 49s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.364521227696728
[ XGBoost ]
Iteration number: 71 50s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.375519569831961
[ XGBoost ]
Iteration number: 72 50s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3729084960892022
[ XGBoost ]
Iteration number: 73 51s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3845258966600975
[ XGBoost ]
Iteration number: 74 51s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3685131130157018
[ XGBoost ]
Iteration number: 75 52s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3613489886733814
[ XGBoost ]
Iteration number: 76 52s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3485967173547906
[ XGBoost ]
Iteration number: 77 53s (1s) 

[ XGBoost ]
Iteration number: 26 24s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1720809682351723
[ XGBoost ]
Iteration number: 27 25s (1s) (91s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2081901605782799
[ XGBoost ]
Iteration number: 28 26s (1s) (91s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1982855007059694
[ XGBoost ]
Iteration number: 29 26s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2276642611255502
[ XGBoost ]
Iteration number: 30 27s (1s) (91s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2281239061944573
[ XGBoost ]
Iteration number: 31 28s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.218221152944553
[ XGBoost ]
Iteration number: 32 29s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2315903846769831
[ XGBoost ]
Iteration number: 33 30s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4985610388852515
[ XGBoost ]
Iteration number: 34 30s (1s)

[ XGBoost ]
Iteration number: 94 133s (1s) (142s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2267109147875925
[ XGBoost ]
Iteration number: 95 135s (1s) (142s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2212140338688997
[ XGBoost ]
Iteration number: 96 137s (1s) (142s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.213047251280863
[ XGBoost ]
Iteration number: 97 139s (1s) (143s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2162983222530208
[ XGBoost ]
Iteration number: 98 141s (1s) (144s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2256744530312251
[ XGBoost ]
Iteration number: 99 143s (1s) (145s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.231627320819994
[ XGBoost ]
Iteration number: 100 146s (1s) (146s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2416824236783193

{'model_list': [<models.classifiers.XGboost object at 0x1a2774c690>], 'explained': '[ *GBoost is an open-source software library which provides th

In [195]:
!rm ../../../AutoPrognosisThings/outputs/agg_CEREBROVASCULAR.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage_CEREBROVASCULAR.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2\
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_CEREBROVASCULAR.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_CEREBROVASCULAR.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

rm: ../../../AutoPrognosisThings/outputs/agg_CEREBROVASCULAR.txt: No such file or directory
[ XGBoost ]
[ XGBoost ]
Iteration number: 1 1s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 1s (1s) (56s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 2s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.995047857909005
[ XGBoost ]
Iteration number: 4 2s (1s) (56s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1042548042232385
[ XGBoost ]
Iteration number: 5 3s (1s) (56s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3299454730716858
[ XGBoost ]
Iteration number: 6 3s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5183108000522332
[ XGBoost ]
Iteration number: 7 4s (1s) (56s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3819021177467412
[ XGBoost ]
Iteration number: 8 4s (1s) (56s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4

[ XGBoost ]
Iteration number: 68 40s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5066750155849231
[ XGBoost ]
Iteration number: 69 41s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5082833040744903
[ XGBoost ]
Iteration number: 70 42s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.502942941015223
[ XGBoost ]
Iteration number: 71 42s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5083964206366778
[ XGBoost ]
Iteration number: 72 43s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.520918829605524
[ XGBoost ]
Iteration number: 73 43s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5212604565839836
[ XGBoost ]
Iteration number: 74 44s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4806631542398543
[ XGBoost ]
Iteration number: 75 45s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4873112506221466
[ XGBoost ]
Iteration number: 76 45s (1s) 

Iteration number: 24 17s (1s) (72s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.384253631325378
[ XGBoost ]
Iteration number: 25 18s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3963709988975035
[ XGBoost ]
Iteration number: 26 18s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4378617428973923
[ XGBoost ]
Iteration number: 27 19s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4381515621779066
[ XGBoost ]
Iteration number: 28 20s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4757325327294388
[ XGBoost ]
Iteration number: 29 20s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4289197700523926
[ XGBoost ]
Iteration number: 30 21s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3900183903781378
[ XGBoost ]
Iteration number: 31 21s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3361401129564954
[ XGBoost ]
Iteration number: 32 22s (1s) (69s), Curr

[ XGBoost ]
Iteration number: 93 62s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7582299626648226
[ XGBoost ]
Iteration number: 94 62s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7578833293730705
[ XGBoost ]
Iteration number: 95 63s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7603915141894384
[ XGBoost ]
Iteration number: 96 64s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7667914009471348
[ XGBoost ]
Iteration number: 97 65s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.772036315443809
[ XGBoost ]
Iteration number: 98 65s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7766116701514902
[ XGBoost ]
Iteration number: 99 66s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.78545438345641
[ XGBoost ]
Iteration number: 100 67s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7886952226874477

{'model_list': [<models.classifiers.XGboo

[ XGBoost ]
Iteration number: 49 66s (1s) (134s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3048436598428816
[ XGBoost ]
Iteration number: 50 67s (1s) (135s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3036121815921724
[ XGBoost ]
Iteration number: 51 68s (1s) (134s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3225487249040084
[ XGBoost ]
Iteration number: 52 69s (1s) (132s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3200689454118568
[ XGBoost ]
Iteration number: 53 69s (1s) (131s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.315208760354637
[ XGBoost ]
Iteration number: 54 71s (1s) (131s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3110766235046691
[ XGBoost ]
Iteration number: 55 71s (1s) (130s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3166983263735266
[ XGBoost ]
Iteration number: 56 72s (1s) (129s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3241423361219893
[ XGBoost ]
Iteration number: 57 

...

[ XGBoost ]
Iteration number: 37 32s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1859660076193432
[ XGBoost ]
Iteration number: 38 33s (1s) (87s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1758682069767306
[ XGBoost ]
Iteration number: 39 34s (1s) (87s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.184401515395447
[ XGBoost ]
Iteration number: 40 35s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2075089018796301
[ XGBoost ]
Iteration number: 41 36s (1s) (89s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2250945924655285
[ XGBoost ]
Iteration number: 42 37s (1s) (89s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.249707794990152
[ XGBoost ]
Iteration number: 43 38s (1s) (89s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2502626193403907
[ XGBoost ]
Iteration number: 44 39s (1s) (89s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2359710759394298
[ XGBoost ]
Iteration number: 45 40s (1s) 

[ XGBoost ]
[ XGBoost ]
Iteration number: 1 1s (1s) (68s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 1s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0000000000000366
[ XGBoost ]
Iteration number: 3 2s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1087667709437619
[ XGBoost ]
Iteration number: 4 3s (1s) (85s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.279686284204999
[ XGBoost ]
Iteration number: 5 4s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8882975260941667
[ XGBoost ]
Iteration number: 6 5s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7973052405838018
[ XGBoost ]
Iteration number: 7 6s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9319886194454332
[ XGBoost ]
Iteration number: 8 7s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0504604296032911
[ XGBoost ]
Iteration number: 9 8s (1s) (89s), Current pipeli

Iteration number: 68 88s (1s) (130s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8800552101505364
[ XGBoost ]
Iteration number: 69 89s (1s) (129s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8869562480779691
[ XGBoost ]
Iteration number: 70 90s (1s) (129s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.894428664850842
[ XGBoost ]
Iteration number: 71 92s (1s) (130s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8904230116240887
[ XGBoost ]
Iteration number: 72 95s (1s) (132s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9044732122902908
[ XGBoost ]
Iteration number: 73 95s (1s) (131s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9151851845906918
[ XGBoost ]
Iteration number: 74 97s (1s) (131s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9197352382517232
[ XGBoost ]
Iteration number: 75 99s (1s) (132s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9256956988182233
[ XGBoost ]
Iteration number: 76 100s (1s) (1

[ XGBoost ]
Iteration number: 24 40s (2s) (166s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0665652944430541
[ XGBoost ]
Iteration number: 25 40s (2s) (162s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0646738437875478
[ XGBoost ]
Iteration number: 26 41s (2s) (159s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.088507178439851
[ XGBoost ]
Iteration number: 27 44s (2s) (163s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0987184040728801
[ XGBoost ]
Iteration number: 28 47s (2s) (168s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0727024632268676
[ XGBoost ]
Iteration number: 29 48s (2s) (166s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0715850544229535
[ XGBoost ]
Iteration number: 30 49s (2s) (163s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0849051827988911
[ XGBoost ]
Iteration number: 31 50s (2s) (160s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0962493228253585
[ XGBoost ]
Iteration number: 32 

[ XGBoost ]
Iteration number: 92 183s (2s) (199s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0883363906662604
[ XGBoost ]
Iteration number: 93 184s (2s) (198s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0919848039229867
[ XGBoost ]
Iteration number: 94 185s (2s) (197s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0869000530213446
[ XGBoost ]
Iteration number: 95 188s (2s) (198s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.092832760884701
[ XGBoost ]
Iteration number: 96 189s (2s) (196s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.092225922956173
[ XGBoost ]
Iteration number: 97 190s (2s) (195s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0975244540238254
[ XGBoost ]
Iteration number: 98 192s (2s) (196s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1031279421753812
[ XGBoost ]
Iteration number: 99 193s (2s) (195s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1065712722059111
[ XGBoost ]
Iteration numb

In [196]:
!rm ../../../AutoPrognosisThings/outputs/agg_CardiacArrhythmias.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage_CardiacArrhythmias.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2\
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_CardiacArrhythmias.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_CardiacArrhythmias.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

rm: ../../../AutoPrognosisThings/outputs/agg_CardiacArrhythmias.txt: No such file or directory
[ XGBoost ]
[ XGBoost ]
Iteration number: 1 2s (2s) (195s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 3s (1s) (126s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 3s (1s) (101s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4007301159453662
[ XGBoost ]
Iteration number: 4 4s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.359207039550037
[ XGBoost ]
Iteration number: 5 4s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0909109516360347
[ XGBoost ]
Iteration number: 6 5s (1s) (78s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2267538774932938
[ XGBoost ]
Iteration number: 7 6s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1664287616643403
[ XGBoost ]
Iteration number: 8 6s (1s) (76s), Current pipelines:  [[[ XGBoost ]]], BO objective

[ XGBoost ]
Iteration number: 68 44s (1s) (65s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2020755458539598
[ XGBoost ]
Iteration number: 69 45s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.216550378348288
[ XGBoost ]
Iteration number: 70 46s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.075517975494646
[ XGBoost ]
Iteration number: 71 47s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.088557477105456
[ XGBoost ]
Iteration number: 72 47s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0800105499576973
[ XGBoost ]
Iteration number: 73 48s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.085115971061732
[ XGBoost ]
Iteration number: 74 49s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0714463152905986
[ XGBoost ]
Iteration number: 75 50s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0578554616681073
[ XGBoost ]
Iteration number: 76 50s (1s) (6

[ XGBoost ]
Iteration number: 25 16s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0541075933636372
[ XGBoost ]
Iteration number: 26 16s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.063841324191477
[ XGBoost ]
Iteration number: 27 17s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0648715657749936
[ XGBoost ]
Iteration number: 28 18s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.005602154072481
[ XGBoost ]
Iteration number: 29 18s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0320148174462545
[ XGBoost ]
Iteration number: 30 19s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.014921623072981
[ XGBoost ]
Iteration number: 31 20s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0074027021690197
[ XGBoost ]
Iteration number: 32 21s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0146668389291023
[ XGBoost ]
Iteration number: 33 21s (1s) (

[ XGBoost ]
Iteration number: 94 69s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.134247941821681
[ XGBoost ]
Iteration number: 95 70s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1372059526948923
[ XGBoost ]
Iteration number: 96 71s (1s) (74s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1442303927381512
[ XGBoost ]
Iteration number: 97 72s (1s) (74s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1425104307881278
[ XGBoost ]
Iteration number: 98 72s (1s) (74s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.135771066060887
[ XGBoost ]
Iteration number: 99 74s (1s) (74s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.128574041391444
[ XGBoost ]
Iteration number: 100 75s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1330319757996636

{'model_list': [<models.classifiers.XGboost object at 0x1a253d66d0>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boos

[ XGBoost ]
Iteration number: 51 40s (1s) (79s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0469627304952636
[ XGBoost ]
Iteration number: 52 41s (1s) (78s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0384277688170804
[ XGBoost ]
Iteration number: 53 41s (1s) (78s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0367026065360632
[ XGBoost ]
Iteration number: 54 42s (1s) (79s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0354861264862711
[ XGBoost ]
Iteration number: 55 43s (1s) (78s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0455520248665018
[ XGBoost ]
Iteration number: 56 44s (1s) (78s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0443359584357432
[ XGBoost ]
Iteration number: 57 45s (1s) (78s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.035161081888745
[ XGBoost ]
Iteration number: 58 45s (1s) (78s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0345696115385314
[ XGBoost ]
Iteration number: 59 46s (1s)

...

[ XGBoost ]
Iteration number: 51 48s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9788736614605337
[ XGBoost ]
Iteration number: 52 49s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9669435274450483
[ XGBoost ]
Iteration number: 53 50s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9705001165966121
[ XGBoost ]
Iteration number: 54 50s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9702724171819179
[ XGBoost ]
Iteration number: 55 51s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9663360987727708
[ XGBoost ]
Iteration number: 56 52s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9708719944932778
[ XGBoost ]
Iteration number: 57 53s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9620299451407097
[ XGBoost ]
Iteration number: 58 54s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9505192595777908
[ XGBoost ]
Iteration number: 59 54s (1s

Iteration number: 7 4s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.73283958606934
[ XGBoost ]
Iteration number: 8 5s (1s) (58s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7382795451997274
[ XGBoost ]
Iteration number: 9 5s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8272273759469858
[ XGBoost ]
Iteration number: 10 6s (1s) (61s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8882494381278292
[ XGBoost ]
Iteration number: 11 7s (1s) (61s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.847005571446932
[ XGBoost ]
Iteration number: 12 7s (1s) (61s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8069345902260286
[ XGBoost ]
Iteration number: 13 8s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7935894392112874
[ XGBoost ]
Iteration number: 14 9s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8506101549254835
[ XGBoost ]
Iteration number: 15 9s (1s) (62s), Current pipelines:

[ XGBoost ]
Iteration number: 76 51s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.020354363546519
[ XGBoost ]
Iteration number: 77 52s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.02217668185492
[ XGBoost ]
Iteration number: 78 52s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0240956112791844
[ XGBoost ]
Iteration number: 79 53s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0250773348759283
[ XGBoost ]
Iteration number: 80 54s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0359817040263668
[ XGBoost ]
Iteration number: 81 55s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0463216887323616
[ XGBoost ]
Iteration number: 82 55s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0468195086398047
[ XGBoost ]
Iteration number: 83 56s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0503796867613087
[ XGBoost ]
Iteration number: 84 57s (1s) (

[ XGBoost ]
Iteration number: 33 23s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8572133827163539
[ XGBoost ]
Iteration number: 34 24s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8696298250451538
[ XGBoost ]
Iteration number: 35 25s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8691773471744427
[ XGBoost ]
Iteration number: 36 26s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8785610747180922
[ XGBoost ]
Iteration number: 37 26s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8640913130716386
[ XGBoost ]
Iteration number: 38 27s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8667121966152395
[ XGBoost ]
Iteration number: 39 27s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8612388245918151
[ XGBoost ]
Iteration number: 40 28s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8527391459270222
[ XGBoost ]
Iteration number: 41 29s (1s

mean: 0.7177630588589492 std: 0.0061844806846274



In [197]:
!rm ../../../AutoPrognosisThings/outputs/agg_CoronaryArtery.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage_CoronaryArtery.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2\
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_CoronaryArtery.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_CoronaryArtery.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

rm: ../../../AutoPrognosisThings/outputs/agg_CoronaryArtery.txt: No such file or directory
[ XGBoost ]
[ XGBoost ]
Iteration number: 1 0s (0s) (44s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 1s (1s) (51s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 2s (1s) (52s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7279406613878409
[ XGBoost ]
Iteration number: 4 2s (1s) (50s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8822303195363922
[ XGBoost ]
Iteration number: 5 2s (0s) (50s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0213936977741063
[ XGBoost ]
Iteration number: 6 3s (1s) (50s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1520464517866755
[ XGBoost ]
Iteration number: 7 6s (1s) (86s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9976935739114705
[ XGBoost ]
Iteration number: 8 10s (1s) (127s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1

[ XGBoost ]
Iteration number: 68 69s (1s) (101s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7938837152821919
[ XGBoost ]
Iteration number: 69 70s (1s) (102s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7914052244275759
[ XGBoost ]
Iteration number: 70 71s (1s) (102s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7966674629491957
[ XGBoost ]
Iteration number: 71 73s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7932864229909614
[ XGBoost ]
Iteration number: 72 74s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7857622044673545
[ XGBoost ]
Iteration number: 73 76s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.783792578200107
[ XGBoost ]
Iteration number: 74 77s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7773435841046721
[ XGBoost ]
Iteration number: 75 78s (1s) (105s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7812720554025606
[ XGBoost ]
Iteration number: 76 

[ XGBoost ]
Iteration number: 24 19s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6521097140350125
[ XGBoost ]
Iteration number: 25 20s (1s) (79s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.674337438849804
[ XGBoost ]
Iteration number: 26 21s (1s) (79s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6886702626198735
[ XGBoost ]
Iteration number: 27 21s (1s) (79s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6779805941815257
[ XGBoost ]
Iteration number: 28 22s (1s) (79s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6783198947597094
[ XGBoost ]
Iteration number: 29 23s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6709351216606211
[ XGBoost ]
Iteration number: 30 24s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6570666738950404
[ XGBoost ]
Iteration number: 31 25s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6503034720253585
[ XGBoost ]
Iteration number: 32 26s (1s)

[ XGBoost ]
Iteration number: 93 97s (1s) (104s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6906917946170936
[ XGBoost ]
Iteration number: 94 99s (1s) (105s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6895645407538589
[ XGBoost ]
Iteration number: 95 100s (1s) (105s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6870669489698078
[ XGBoost ]
Iteration number: 96 102s (1s) (106s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6888069978311901
[ XGBoost ]
Iteration number: 97 103s (1s) (106s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6884627768372162
[ XGBoost ]
Iteration number: 98 104s (1s) (106s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.69233873171491
[ XGBoost ]
Iteration number: 99 106s (1s) (107s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6902611497216092
[ XGBoost ]
Iteration number: 100 107s (1s) (107s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6901979054366466

{'model_list': [<models.cl

Iteration number: 49 29s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5658060846364568
[ XGBoost ]
Iteration number: 50 29s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5747176851152433
[ XGBoost ]
Iteration number: 51 30s (1s) (58s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5607927522481984
[ XGBoost ]
Iteration number: 52 30s (1s) (58s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5718333823332467
[ XGBoost ]
Iteration number: 53 31s (1s) (58s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5731384133163422
[ XGBoost ]
Iteration number: 54 31s (1s) (58s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5432351455746065
[ XGBoost ]
Iteration number: 55 32s (1s) (58s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5530666716878478
[ XGBoost ]
Iteration number: 56 33s (1s) (58s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5182745471231172
[ XGBoost ]
Iteration number: 57 33s (1s) (58s), Cur

...

Iteration number: 37 26s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6329494100470069
[ XGBoost ]
Iteration number: 38 26s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6335626797538563
[ XGBoost ]
Iteration number: 39 27s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6270565668534338
[ XGBoost ]
Iteration number: 40 27s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6178433741171099
[ XGBoost ]
Iteration number: 41 28s (1s) (68s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6374090610978927
[ XGBoost ]
Iteration number: 42 29s (1s) (68s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6527910768697088
[ XGBoost ]
Iteration number: 43 30s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6595001224218977
[ XGBoost ]
Iteration number: 44 30s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6587984051783847
[ XGBoost ]
Iteration number: 45 31s (1s) (69s), Cur

[ XGBoost ]
[ XGBoost ]
Iteration number: 1 1s (1s) (134s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 3s (1s) (134s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.999999999999988
[ XGBoost ]
Iteration number: 3 7s (2s) (232s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8094667719064006
[ XGBoost ]
Iteration number: 4 8s (2s) (190s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7153003035444216
[ XGBoost ]
Iteration number: 5 8s (2s) (166s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7289550008505365
[ XGBoost ]
Iteration number: 6 9s (1s) (148s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6264933636399477
[ XGBoost ]
Iteration number: 7 9s (1s) (134s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6361519592146524
[ XGBoost ]
Iteration number: 8 13s (2s) (164s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7532482527811316
[ XGBoost ]
Iteration number: 9 14s (2s) (154s), Cur

[ XGBoost ]
Iteration number: 68 69s (1s) (102s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7677055361884392
[ XGBoost ]
Iteration number: 69 70s (1s) (102s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7629456342771797
[ XGBoost ]
Iteration number: 70 72s (1s) (102s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7721403957736033
[ XGBoost ]
Iteration number: 71 74s (1s) (104s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7688065349756745
[ XGBoost ]
Iteration number: 72 75s (1s) (104s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7714437151906561
[ XGBoost ]
Iteration number: 73 75s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7645336950828426
[ XGBoost ]
Iteration number: 74 76s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7736578488216641
[ XGBoost ]
Iteration number: 75 77s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7722444464231918
[ XGBoost ]
Iteration number: 76

[ XGBoost ]
Iteration number: 24 20s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7064066314201877
[ XGBoost ]
Iteration number: 25 20s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7145393068587902
[ XGBoost ]
Iteration number: 26 21s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6794821922287083
[ XGBoost ]
Iteration number: 27 22s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.696670564959113
[ XGBoost ]
Iteration number: 28 23s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7021613599386074
[ XGBoost ]
Iteration number: 29 23s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6963414651028849
[ XGBoost ]
Iteration number: 30 27s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7054896506765156
[ XGBoost ]
Iteration number: 31 28s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7165272301866679
[ XGBoost ]
Iteration number: 32 29s (1s)

[ XGBoost ]
Iteration number: 92 92s (1s) (100s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7434984243615025
[ XGBoost ]
Iteration number: 93 93s (1s) (100s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7436602966719656
[ XGBoost ]
Iteration number: 94 94s (1s) (100s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7398405928176333
[ XGBoost ]
Iteration number: 95 95s (1s) (100s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7438752886484705
[ XGBoost ]
Iteration number: 96 96s (1s) (100s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7395008180976304
[ XGBoost ]
Iteration number: 97 97s (1s) (100s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7388209978029219
[ XGBoost ]
Iteration number: 98 98s (1s) (100s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7487384304952458
[ XGBoost ]
Iteration number: 99 99s (1s) (100s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.750150101033556
[ XGBoost ]
Iteration number: 100

### image probability included

In [198]:
!rm ../../../AutoPrognosisThings/outputs/agg_green_with_prob.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_green.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2 \
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_green_with_prob.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_green_with_prob.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

rm: ../../../AutoPrognosisThings/outputs/agg_green_with_prob.txt: No such file or directory
[ XGBoost ]
[ XGBoost ]
Iteration number: 1 0s (0s) (49s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 1s (0s) (46s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 1s (0s) (46s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.410164819434139
[ XGBoost ]
Iteration number: 4 2s (0s) (49s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9603939297150078
[ XGBoost ]
Iteration number: 5 3s (1s) (51s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8963831286854832
[ XGBoost ]
Iteration number: 6 3s (0s) (50s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9433641789789943
[ XGBoost ]
Iteration number: 7 4s (1s) (50s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0404921696954692
[ XGBoost ]
Iteration number: 8 4s (1s) (51s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1

[ XGBoost ]
Iteration number: 68 43s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5834505791696953
[ XGBoost ]
Iteration number: 69 44s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5774513229977336
[ XGBoost ]
Iteration number: 70 45s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5809138550912187
[ XGBoost ]
Iteration number: 71 45s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5964410333652748
[ XGBoost ]
Iteration number: 72 46s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6118606181804291
[ XGBoost ]
Iteration number: 73 46s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5742289879433526
[ XGBoost ]
Iteration number: 74 47s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5560683186202495
[ XGBoost ]
Iteration number: 75 48s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.543011599419788
[ XGBoost ]
Iteration number: 76 48s (1s)

[ XGBoost ]
Iteration number: 25 15s (1s) (61s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2906055230500315
[ XGBoost ]
Iteration number: 26 16s (1s) (61s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2958267603504139
[ XGBoost ]
Iteration number: 27 16s (1s) (61s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2848962342408825
[ XGBoost ]
Iteration number: 28 17s (1s) (61s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2593900901590298
[ XGBoost ]
Iteration number: 29 17s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2343845165450065
[ XGBoost ]
Iteration number: 30 18s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2496110677673007
[ XGBoost ]
Iteration number: 31 19s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2666347642656683
[ XGBoost ]
Iteration number: 32 19s (1s) (61s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2639894493119848
[ XGBoost ]
Iteration number: 33 20s (1s

[ XGBoost ]
Iteration number: 94 58s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2925866917222704
[ XGBoost ]
Iteration number: 95 59s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2978371436345686
[ XGBoost ]
Iteration number: 96 60s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3029985963436774
[ XGBoost ]
Iteration number: 97 60s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.304842815257594
[ XGBoost ]
Iteration number: 98 61s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3034492553600592
[ XGBoost ]
Iteration number: 99 62s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.311764969228478
[ XGBoost ]
Iteration number: 100 62s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4362720732934082

{'model_list': [<models.classifiers.XGboost object at 0x1a1d6ee750>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boo

[ XGBoost ]
Iteration number: 51 49s (1s) (96s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2917509365543232
[ XGBoost ]
Iteration number: 52 51s (1s) (97s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2926698409878081
[ XGBoost ]
Iteration number: 53 51s (1s) (97s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.311545160410866
[ XGBoost ]
Iteration number: 54 52s (1s) (96s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.308946309611142
[ XGBoost ]
Iteration number: 55 52s (1s) (95s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3279706375548275
[ XGBoost ]
Iteration number: 56 53s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3394872422259843
[ XGBoost ]
Iteration number: 57 55s (1s) (96s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3577336498995496
[ XGBoost ]
Iteration number: 58 55s (1s) (95s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3494429513264035
[ XGBoost ]
Iteration number: 59 56s (1s) 

...

[ XGBoost ]
Iteration number: 50 44s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3848179077561271
[ XGBoost ]
Iteration number: 51 45s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3673359309750392
[ XGBoost ]
Iteration number: 52 46s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3416179348002963
[ XGBoost ]
Iteration number: 53 47s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3342384498981923
[ XGBoost ]
Iteration number: 54 48s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.33360821503012
[ XGBoost ]
Iteration number: 55 48s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3138770793137937
[ XGBoost ]
Iteration number: 56 49s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.305277576768802
[ XGBoost ]
Iteration number: 57 50s (1s) (88s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.291758188683627
[ XGBoost ]
Iteration number: 58 51s (1s) (8

[ XGBoost ]
Iteration number: 7 7s (1s) (106s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0981513215558365
[ XGBoost ]
Iteration number: 8 8s (1s) (98s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2271534829593904
[ XGBoost ]
Iteration number: 9 9s (1s) (97s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2572011751801226
[ XGBoost ]
Iteration number: 10 9s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3483291259077
[ XGBoost ]
Iteration number: 11 10s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.362369369770672
[ XGBoost ]
Iteration number: 12 12s (1s) (99s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.371242255343269
[ XGBoost ]
Iteration number: 13 14s (1s) (110s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4116460792480365
[ XGBoost ]
Iteration number: 14 15s (1s) (106s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4101882843189126
[ XGBoost ]
Iteration number: 15 16s (1s) (103s),

[ XGBoost ]
Iteration number: 76 71s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3862132359929278
[ XGBoost ]
Iteration number: 77 71s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3757370337659696
[ XGBoost ]
Iteration number: 78 72s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3759006544117978
[ XGBoost ]
Iteration number: 79 73s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.36234425827692
[ XGBoost ]
Iteration number: 80 74s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.374078314156234
[ XGBoost ]
Iteration number: 81 75s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3612831226066626
[ XGBoost ]
Iteration number: 82 75s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3447512791718115
[ XGBoost ]
Iteration number: 83 76s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3385468349287968
[ XGBoost ]
Iteration number: 84 77s (1s) (

[ XGBoost ]
Iteration number: 33 30s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2187802565612706
[ XGBoost ]
Iteration number: 34 34s (1s) (100s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1921920912214303
[ XGBoost ]
Iteration number: 35 35s (1s) (99s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1686418952797368
[ XGBoost ]
Iteration number: 36 35s (1s) (98s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1893404607622697
[ XGBoost ]
Iteration number: 37 36s (1s) (96s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1929351018361654
[ XGBoost ]
Iteration number: 38 37s (1s) (96s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1611903386481677
[ XGBoost ]
Iteration number: 39 39s (1s) (99s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.144960834403655
[ XGBoost ]
Iteration number: 40 39s (1s) (98s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.166881454082076
[ XGBoost ]
Iteration number: 41 40s (1s)

mean: 0.6973496365983178 std: 0.004267475033398126



In [218]:
array1 = []
with open('../../../AutoPrognosisThings/outputs/agg_green_with_prob.txt') as f:
    for i in range(10):
        array1.append([float(x) for x in next(f).split()])
array2 = []
with open('../../../AutoPrognosisThings/outputs/agg_green.txt') as f:
    for i in range(10):
        array2.append([float(x) for x in next(f).split()])

result= stats.ttest_ind(array1, array2)
result

Ttest_indResult(statistic=array([-2.28488526]), pvalue=array([0.03467156]))

In [215]:
!rm ../../../AutoPrognosisThings/outputs/agg_orange_with_prob.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_orange.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2\
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_orange_with_prob.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_orange_with_prob.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

[ XGBoost ]
[ XGBoost ]
Iteration number: 1 1s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 1s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 2s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3699880125529442
[ XGBoost ]
Iteration number: 4 5s (1s) (120s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5568093021134748
[ XGBoost ]
Iteration number: 5 5s (1s) (108s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3141065743388842
[ XGBoost ]
Iteration number: 6 6s (1s) (101s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1844845678665872
[ XGBoost ]
Iteration number: 7 7s (1s) (96s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0794749727734165
[ XGBoost ]
Iteration number: 8 7s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0151387925039093
[ XGBoost ]
Iteration number: 9 8s (1s) (91s), Current pipelines:  [[[ X

[ XGBoost ]
Iteration number: 69 50s (1s) (72s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2608760930106708
[ XGBoost ]
Iteration number: 70 50s (1s) (72s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2735830245546391
[ XGBoost ]
Iteration number: 71 51s (1s) (72s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2579809364185295
[ XGBoost ]
Iteration number: 72 52s (1s) (72s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.249425507227297
[ XGBoost ]
Iteration number: 73 52s (1s) (72s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2435576926242704
[ XGBoost ]
Iteration number: 74 53s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2346891403701343
[ XGBoost ]
Iteration number: 75 54s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.23208121177289
[ XGBoost ]
Iteration number: 76 54s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2239458238513687
[ XGBoost ]
Iteration number: 77 55s (1s) (

[ XGBoost ]
Iteration number: 26 17s (1s) (65s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1288128122310992
[ XGBoost ]
Iteration number: 27 17s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1664906676613283
[ XGBoost ]
Iteration number: 28 18s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1623818364117562
[ XGBoost ]
Iteration number: 29 18s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1360416203944268
[ XGBoost ]
Iteration number: 30 19s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.562203674916097
[ XGBoost ]
Iteration number: 31 19s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5482888603331062
[ XGBoost ]
Iteration number: 32 20s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.547571184636981
[ XGBoost ]
Iteration number: 33 20s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5498733535631337
[ XGBoost ]
Iteration number: 34 21s (1s) 

[ XGBoost ]
Iteration number: 95 60s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3448771786046434
[ XGBoost ]
Iteration number: 96 61s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.347024071080463
[ XGBoost ]
Iteration number: 97 62s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3521770059850158
[ XGBoost ]
Iteration number: 98 62s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.355702494108856
[ XGBoost ]
Iteration number: 99 63s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3564186701709688
[ XGBoost ]
Iteration number: 100 64s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3658734880223902

{'model_list': [<models.classifiers.XGboost object at 0x1a1ca1f750>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes': None, 'num_stages': 1, 'pipelin

[ XGBoost ]
Iteration number: 52 44s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2954733194182577
[ XGBoost ]
Iteration number: 53 45s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2795431084959699
[ XGBoost ]
Iteration number: 54 45s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2888879632287589
[ XGBoost ]
Iteration number: 55 46s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.294435742322379
[ XGBoost ]
Iteration number: 56 47s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2975974494721139
[ XGBoost ]
Iteration number: 57 48s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3110185000713546
[ XGBoost ]
Iteration number: 58 48s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.324423122724699
[ XGBoost ]
Iteration number: 59 49s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3157556578557792
[ XGBoost ]
Iteration number: 60 50s (1s) 

...

[ XGBoost ]
Iteration number: 37 37s (1s) (99s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4551308627750552
[ XGBoost ]
Iteration number: 38 38s (1s) (100s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4582815738936508
[ XGBoost ]
Iteration number: 39 39s (1s) (101s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4704403519524973
[ XGBoost ]
Iteration number: 40 41s (1s) (102s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.468836731885931
[ XGBoost ]
Iteration number: 41 42s (1s) (103s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4720758138483017
[ XGBoost ]
Iteration number: 42 44s (1s) (104s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4761036521030426
[ XGBoost ]
Iteration number: 43 45s (1s) (105s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4735865808202826
[ XGBoost ]
Iteration number: 44 47s (1s) (106s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4759972140294104
[ XGBoost ]
Iteration number: 45 4

[ XGBoost ]
[ XGBoost ]
Iteration number: 1 2s (2s) (191s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 2s (1s) (118s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 3s (1s) (98s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9281559231116681
[ XGBoost ]
Iteration number: 4 3s (1s) (87s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1051488447089124
[ XGBoost ]
Iteration number: 5 4s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.326363438767855
[ XGBoost ]
Iteration number: 6 5s (1s) (79s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5138930415348373
[ XGBoost ]
Iteration number: 7 6s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.413098959238251
[ XGBoost ]
Iteration number: 8 7s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3331114516242069
[ XGBoost ]
Iteration number: 9 8s (1s) (84s), Current pipelines:  [[[ XGBo

[ XGBoost ]
Iteration number: 69 56s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4662302727453413
[ XGBoost ]
Iteration number: 70 57s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4721400443581818
[ XGBoost ]
Iteration number: 71 58s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4700211938181953
[ XGBoost ]
Iteration number: 72 58s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4708494097296287
[ XGBoost ]
Iteration number: 73 59s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4775015361928021
[ XGBoost ]
Iteration number: 74 60s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4812662473530214
[ XGBoost ]
Iteration number: 75 60s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4623601145670957
[ XGBoost ]
Iteration number: 76 61s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4671484091080906
[ XGBoost ]
Iteration number: 77 62s (1s

[ XGBoost ]
Iteration number: 26 17s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.266881466161817
[ XGBoost ]
Iteration number: 27 17s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3051926094409376
[ XGBoost ]
Iteration number: 28 18s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.342899494038902
[ XGBoost ]
Iteration number: 29 18s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.309959805137298
[ XGBoost ]
Iteration number: 30 19s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3267350486828466
[ XGBoost ]
Iteration number: 31 19s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2910402109088872
[ XGBoost ]
Iteration number: 32 20s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.313963925455606
[ XGBoost ]
Iteration number: 33 21s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.266401672645566
[ XGBoost ]
Iteration number: 34 21s (1s) (63

[ XGBoost ]
Iteration number: 95 62s (1s) (65s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2706389139786012
[ XGBoost ]
Iteration number: 96 63s (1s) (65s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2656750614780692
[ XGBoost ]
Iteration number: 97 63s (1s) (65s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2700289338929436
[ XGBoost ]
Iteration number: 98 64s (1s) (65s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2586943300713989
[ XGBoost ]
Iteration number: 99 65s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.280554007772578
[ XGBoost ]
Iteration number: 100 66s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2824541596937595

{'model_list': [<models.classifiers.XGboost object at 0x1a2446d710>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes': None, 'num_stages': 1, 'pipeli

In [219]:
array1 = []
with open('../../../AutoPrognosisThings/outputs/agg_orange_with_prob.txt') as f:
    for i in range(10):
        array1.append([float(x) for x in next(f).split()])
array2 = []
with open('../../../AutoPrognosisThings/outputs/agg_orange.txt') as f:
    for i in range(10):
        array2.append([float(x) for x in next(f).split()])

result= stats.ttest_ind(array1, array2)
result

Ttest_indResult(statistic=array([-3.17210826]), pvalue=array([0.00527608]))

In [200]:
!rm ../../../AutoPrognosisThings/outputs/agg_vascular_with_prob.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_vascular.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2\
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_vascular_with_prob.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_vascular_with_prob.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

rm: ../../../AutoPrognosisThings/outputs/agg_vascular_with_prob.txt: No such file or directory
[ XGBoost ]
[ XGBoost ]
Iteration number: 1 0s (0s) (39s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 1s (0s) (42s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 2s (1s) (53s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.199819817967807
[ XGBoost ]
Iteration number: 4 3s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3011154710815034
[ XGBoost ]
Iteration number: 5 4s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1119087491179307
[ XGBoost ]
Iteration number: 6 4s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1268285298868288
[ XGBoost ]
Iteration number: 7 5s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.210641370859144
[ XGBoost ]
Iteration number: 8 5s (1s) (68s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1

[ XGBoost ]
Iteration number: 68 39s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6787028696255488
[ XGBoost ]
Iteration number: 69 39s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.68473213356712
[ XGBoost ]
Iteration number: 70 40s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.683194186988106
[ XGBoost ]
Iteration number: 71 41s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6916214818778383
[ XGBoost ]
Iteration number: 72 41s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.704991479392925
[ XGBoost ]
Iteration number: 73 42s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6987276259403294
[ XGBoost ]
Iteration number: 74 42s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.714276053524139
[ XGBoost ]
Iteration number: 75 43s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6848415462475812
[ XGBoost ]
Iteration number: 76 43s (1s) (57

[ XGBoost ]
Iteration number: 25 18s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.341026785842677
[ XGBoost ]
Iteration number: 26 19s (1s) (74s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3281878480356677
[ XGBoost ]
Iteration number: 27 20s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2749036102018936
[ XGBoost ]
Iteration number: 28 20s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2813277361987438
[ XGBoost ]
Iteration number: 29 21s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2778046374067795
[ XGBoost ]
Iteration number: 30 22s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2756556630922335
[ XGBoost ]
Iteration number: 31 23s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2732156963228873
[ XGBoost ]
Iteration number: 32 23s (1s) (72s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2548596048821143
[ XGBoost ]
Iteration number: 33 24s (1s)

[ XGBoost ]
Iteration number: 94 68s (1s) (72s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4453263191599126
[ XGBoost ]
Iteration number: 95 70s (1s) (74s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.429809742874062
[ XGBoost ]
Iteration number: 96 71s (1s) (74s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4389414651312193
[ XGBoost ]
Iteration number: 97 72s (1s) (74s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.447286227931426
[ XGBoost ]
Iteration number: 98 74s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4536593183903284
[ XGBoost ]
Iteration number: 99 75s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4607936341162158
[ XGBoost ]
Iteration number: 100 75s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4704119186628586

{'model_list': [<models.classifiers.XGboost object at 0x1a1a770710>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boo

[ XGBoost ]
Iteration number: 51 48s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.8142442607554234
[ XGBoost ]
Iteration number: 52 49s (1s) (95s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.8061276428134243
[ XGBoost ]
Iteration number: 53 51s (1s) (96s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7863554757496665
[ XGBoost ]
Iteration number: 54 52s (1s) (96s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7674508127135593
[ XGBoost ]
Iteration number: 55 52s (1s) (95s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7796090167186729
[ XGBoost ]
Iteration number: 56 54s (1s) (96s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7775554958511885
[ XGBoost ]
Iteration number: 57 54s (1s) (95s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7932098680346122
[ XGBoost ]
Iteration number: 58 55s (1s) (95s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.8129583315782718
[ XGBoost ]
Iteration number: 59 56s (1s

...

[ XGBoost ]
Iteration number: 50 28s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3321175987403822
[ XGBoost ]
Iteration number: 51 29s (1s) (56s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3206355948262585
[ XGBoost ]
Iteration number: 52 29s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.327101670800505
[ XGBoost ]
Iteration number: 53 30s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3160626050020456
[ XGBoost ]
Iteration number: 54 31s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3271749278140879
[ XGBoost ]
Iteration number: 55 31s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3458696763675067
[ XGBoost ]
Iteration number: 56 32s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3558863992703136
[ XGBoost ]
Iteration number: 57 32s (1s) (57s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.373191986596761
[ XGBoost ]
Iteration number: 58 33s (1s) 

[ XGBoost ]
Iteration number: 7 4s (1s) (54s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2262810825612587
[ XGBoost ]
Iteration number: 8 4s (1s) (53s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3575087660432656
[ XGBoost ]
Iteration number: 9 5s (1s) (54s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4783463822505134
[ XGBoost ]
Iteration number: 10 5s (1s) (54s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.568695512210021
[ XGBoost ]
Iteration number: 11 6s (1s) (53s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5508829756081446
[ XGBoost ]
Iteration number: 12 9s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6396998633770083
[ XGBoost ]
Iteration number: 13 10s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6443571141367874
[ XGBoost ]
Iteration number: 14 10s (1s) (72s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6186583274197497
[ XGBoost ]
Iteration number: 15 11s (1s) (71s), C

[ XGBoost ]
Iteration number: 76 100s (1s) (131s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.437421029432253
[ XGBoost ]
Iteration number: 77 102s (1s) (133s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4191298125715661
[ XGBoost ]
Iteration number: 78 103s (1s) (132s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.431545323053659
[ XGBoost ]
Iteration number: 79 104s (1s) (132s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5685382634769403
[ XGBoost ]
Iteration number: 80 105s (1s) (131s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5646578831331472
[ XGBoost ]
Iteration number: 81 106s (1s) (130s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5524910245270938
[ XGBoost ]
Iteration number: 82 106s (1s) (130s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5571466725505185
[ XGBoost ]
Iteration number: 83 110s (1s) (133s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5443212353261035
[ XGBoost ]
Iteration numb

[ XGBoost ]
Iteration number: 32 19s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2219922115446147
[ XGBoost ]
Iteration number: 33 20s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2067041778576464
[ XGBoost ]
Iteration number: 34 20s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1856020147231978
[ XGBoost ]
Iteration number: 35 21s (1s) (59s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1861190449281866
[ XGBoost ]
Iteration number: 36 22s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1632002633308811
[ XGBoost ]
Iteration number: 37 22s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1605389218207398
[ XGBoost ]
Iteration number: 38 23s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1262399936170344
[ XGBoost ]
Iteration number: 39 24s (1s) (61s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1375235052127333
[ XGBoost ]
Iteration number: 40 25s (1s

{'model_list': [<models.classifiers.XGboost object at 0x1a17d09790>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes': None, 'num_stages': 1, 'pipeline_stages': ['classifier'], 'name': '[ XGBoost ]', 'analysis_mode': None, 'analysis_type': None}
{'model_list': [<models.classifiers.XGboost object at 0x1a17d09850>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes': None, 'num_stages': 1, 'pipeline_stages': ['classifier'], 'name': '[ XGBoost ]', 'analysis_mode': None, 'analysis_type': None}
{'model_list': [<models.classifiers.XGboost object at 0x1a17d09950>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes':

In [220]:
array1 = []
with open('../../../AutoPrognosisThings/outputs/agg_vascular_with_prob.txt') as f:
    for i in range(10):
        array1.append([float(x) for x in next(f).split()])
array2 = []
with open('../../../AutoPrognosisThings/outputs/agg_vascular.txt') as f:
    for i in range(10):
        array2.append([float(x) for x in next(f).split()])

result= stats.ttest_ind(array1, array2)
result

Ttest_indResult(statistic=array([0.69913599]), pvalue=array([0.493403]))

In [201]:
!rm ../../../AutoPrognosisThings/outputs/agg_cereb_with_prob.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_cereb.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2\
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_cereb_with_prob.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_cereb_with_prob.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

rm: ../../../AutoPrognosisThings/outputs/agg_cereb_with_prob.txt: No such file or directory
[ XGBoost ]
[ XGBoost ]
Iteration number: 1 1s (1s) (55s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 1s (1s) (54s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 2s (1s) (53s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2557337358790437
[ XGBoost ]
Iteration number: 4 2s (1s) (53s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2419185025811266
[ XGBoost ]
Iteration number: 5 3s (1s) (52s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2623206942679899
[ XGBoost ]
Iteration number: 6 3s (1s) (50s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4433425149849766
[ XGBoost ]
Iteration number: 7 3s (0s) (50s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5328321258228879
[ XGBoost ]
Iteration number: 8 4s (1s) (51s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.

[ XGBoost ]
Iteration number: 68 42s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2936544389589277
[ XGBoost ]
Iteration number: 69 43s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2885216507821249
[ XGBoost ]
Iteration number: 70 43s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2929068844142872
[ XGBoost ]
Iteration number: 71 44s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.293517507149711
[ XGBoost ]
Iteration number: 72 45s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.305209881312547
[ XGBoost ]
Iteration number: 73 45s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3095876035820482
[ XGBoost ]
Iteration number: 74 46s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.317664496274661
[ XGBoost ]
Iteration number: 75 47s (1s) (62s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3196184603817438
[ XGBoost ]
Iteration number: 76 47s (1s) (

[ XGBoost ]
Iteration number: 25 17s (1s) (68s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3755969097602938
[ XGBoost ]
Iteration number: 26 17s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3670769451025326
[ XGBoost ]
Iteration number: 27 18s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3806157155364809
[ XGBoost ]
Iteration number: 28 19s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4110965671393454
[ XGBoost ]
Iteration number: 29 19s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4266767207840436
[ XGBoost ]
Iteration number: 30 20s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4319359118010075
[ XGBoost ]
Iteration number: 31 20s (1s) (65s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4206302905660233
[ XGBoost ]
Iteration number: 32 21s (1s) (65s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.399461130208647
[ XGBoost ]
Iteration number: 33 22s (1s)

[ XGBoost ]
Iteration number: 94 66s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3187791650966407
[ XGBoost ]
Iteration number: 95 67s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.319246511783614
[ XGBoost ]
Iteration number: 96 68s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.326195031757515
[ XGBoost ]
Iteration number: 97 69s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3199679895240004
[ XGBoost ]
Iteration number: 98 70s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.326801468835168
[ XGBoost ]
Iteration number: 99 70s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3304792283380653
[ XGBoost ]
Iteration number: 100 71s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.334893681850358

{'model_list': [<models.classifiers.XGboost object at 0x1a1821b710>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boost

[ XGBoost ]
Iteration number: 51 47s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5682112005558546
[ XGBoost ]
Iteration number: 52 49s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.566432887631923
[ XGBoost ]
Iteration number: 53 49s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5744145539228844
[ XGBoost ]
Iteration number: 54 50s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5580601069679527
[ XGBoost ]
Iteration number: 55 50s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.542100593034056
[ XGBoost ]
Iteration number: 56 51s (1s) (91s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5417205552621325
[ XGBoost ]
Iteration number: 57 52s (1s) (91s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5496796882465687
[ XGBoost ]
Iteration number: 58 52s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5261942442404453
[ XGBoost ]
Iteration number: 59 53s (1s) 

...

[ XGBoost ]
Iteration number: 50 32s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8826041320926146
[ XGBoost ]
Iteration number: 51 32s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8915174136056098
[ XGBoost ]
Iteration number: 52 33s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9019229905515209
[ XGBoost ]
Iteration number: 53 34s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8923153439990703
[ XGBoost ]
Iteration number: 54 34s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8947207210879755
[ XGBoost ]
Iteration number: 55 35s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8906407157666137
[ XGBoost ]
Iteration number: 56 35s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.883112932669896
[ XGBoost ]
Iteration number: 57 36s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8711428452322441
[ XGBoost ]
Iteration number: 58 37s (1s)

[ XGBoost ]
Iteration number: 7 5s (1s) (68s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.765308576411069
[ XGBoost ]
Iteration number: 8 5s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.8916485339098008
[ XGBoost ]
Iteration number: 9 6s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.9373656940655408
[ XGBoost ]
Iteration number: 10 7s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.643260538736651
[ XGBoost ]
Iteration number: 11 7s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7165100325266531
[ XGBoost ]
Iteration number: 12 11s (1s) (89s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6640181157900362
[ XGBoost ]
Iteration number: 13 11s (1s) (85s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5946705592008708
[ XGBoost ]
Iteration number: 14 12s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6078619070680908
[ XGBoost ]
Iteration number: 15 15s (1s) (99s), C

[ XGBoost ]
Iteration number: 76 68s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4116915201601166
[ XGBoost ]
Iteration number: 77 69s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4136936474509798
[ XGBoost ]
Iteration number: 78 70s (1s) (89s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.398155119437877
[ XGBoost ]
Iteration number: 79 70s (1s) (89s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.396488912320141
[ XGBoost ]
Iteration number: 80 71s (1s) (89s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3826731073995624
[ XGBoost ]
Iteration number: 81 72s (1s) (89s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3837777397353725
[ XGBoost ]
Iteration number: 82 73s (1s) (89s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.391790355391538
[ XGBoost ]
Iteration number: 83 75s (1s) (90s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6749633526765317
[ XGBoost ]
Iteration number: 84 75s (1s) (

[ XGBoost ]
Iteration number: 33 27s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.210933151474026
[ XGBoost ]
Iteration number: 34 28s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2013232707294157
[ XGBoost ]
Iteration number: 35 29s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2307890184394277
[ XGBoost ]
Iteration number: 36 29s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2226425563360903
[ XGBoost ]
Iteration number: 37 30s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2061003129684371
[ XGBoost ]
Iteration number: 38 31s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2332874027037153
[ XGBoost ]
Iteration number: 39 31s (1s) (80s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.219065429915868
[ XGBoost ]
Iteration number: 40 32s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1924139714957276
[ XGBoost ]
Iteration number: 41 34s (1s) 

mean: 0.7253660296816099 std: 0.011698009196243364



In [222]:
array1 = []
with open('../../../AutoPrognosisThings/outputs/agg_cereb_with_prob.txt') as f:
    for i in range(10):
        array1.append([float(x) for x in next(f).split()])
array2 = []
with open('../../../AutoPrognosisThings/outputs/agg_CEREBROVASCULAR.txt') as f:
    for i in range(10):
        array2.append([float(x) for x in next(f).split()])

result= stats.ttest_ind(array1, array2)
result

Ttest_indResult(statistic=array([0.46919423]), pvalue=array([0.64456392]))

In [202]:
!rm ../../../AutoPrognosisThings/outputs/agg_CardiacArrhythmias_with_prob.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_CardiacArrhythmias.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2\
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_CardiacArrhythmias_with_prob.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_CardiacArrhythmias_with_prob.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

[ XGBoost ]
[ XGBoost ]
Iteration number: 1 0s (0s) (43s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 1s (0s) (42s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 1s (0s) (46s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1852682243719563
[ XGBoost ]
Iteration number: 4 2s (0s) (50s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.131055531485201
[ XGBoost ]
Iteration number: 5 3s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3006237407371348
[ XGBoost ]
Iteration number: 6 4s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.4763460357659977
[ XGBoost ]
Iteration number: 7 5s (1s) (69s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5540403289976183
[ XGBoost ]
Iteration number: 8 5s (1s) (66s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7000021982864681
[ XGBoost ]
Iteration number: 9 6s (1s) (64s), Current pipelines:  [[[ XGBoo

[ XGBoost ]
Iteration number: 69 44s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3003055625311661
[ XGBoost ]
Iteration number: 70 45s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2991157179844186
[ XGBoost ]
Iteration number: 71 45s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2903547860241515
[ XGBoost ]
Iteration number: 72 46s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.289839947321028
[ XGBoost ]
Iteration number: 73 47s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2808166029618964
[ XGBoost ]
Iteration number: 74 47s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2653497523098838
[ XGBoost ]
Iteration number: 75 48s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.266968937237301
[ XGBoost ]
Iteration number: 76 49s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.257911882127004
[ XGBoost ]
Iteration number: 77 49s (1s) (

[ XGBoost ]
Iteration number: 26 16s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1914449395028386
[ XGBoost ]
Iteration number: 27 17s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1792699464384868
[ XGBoost ]
Iteration number: 28 18s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1393667899304947
[ XGBoost ]
Iteration number: 29 18s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.13361227311529
[ XGBoost ]
Iteration number: 30 19s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1430592592525304
[ XGBoost ]
Iteration number: 31 19s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1478491699806843
[ XGBoost ]
Iteration number: 32 20s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.124595988112165
[ XGBoost ]
Iteration number: 33 21s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1236589680994609
[ XGBoost ]
Iteration number: 34 22s (1s) (

[ XGBoost ]
Iteration number: 95 71s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1028707279645562
[ XGBoost ]
Iteration number: 96 72s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.102990373737653
[ XGBoost ]
Iteration number: 97 73s (1s) (76s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1035830332254144
[ XGBoost ]
Iteration number: 98 74s (1s) (76s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1008738508926317
[ XGBoost ]
Iteration number: 99 76s (1s) (76s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0993964523470707
[ XGBoost ]
Iteration number: 100 77s (1s) (77s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1037338398917693

{'model_list': [<models.classifiers.XGboost object at 0x1a211d3750>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes': None, 'num_stages': 1, 'pipeli

[ XGBoost ]
Iteration number: 52 43s (1s) (83s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.342460082219108
[ XGBoost ]
Iteration number: 53 44s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3212752680213438
[ XGBoost ]
Iteration number: 54 44s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.316183740570976
[ XGBoost ]
Iteration number: 55 45s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2976496390427024
[ XGBoost ]
Iteration number: 56 46s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2961898651433013
[ XGBoost ]
Iteration number: 57 47s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2774236099064962
[ XGBoost ]
Iteration number: 58 47s (1s) (82s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.290269597855068
[ XGBoost ]
Iteration number: 59 48s (1s) (81s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.272039593110632
[ XGBoost ]
Iteration number: 60 49s (1s) (8

...

[ XGBoost ]
Iteration number: 51 51s (1s) (99s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5631904761722761
[ XGBoost ]
Iteration number: 52 51s (1s) (98s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5574559882002412
[ XGBoost ]
Iteration number: 53 53s (1s) (99s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5750306636662386
[ XGBoost ]
Iteration number: 54 53s (1s) (99s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5716674848864847
[ XGBoost ]
Iteration number: 55 54s (1s) (98s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5847712262295157
[ XGBoost ]
Iteration number: 56 55s (1s) (98s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.592427536920098
[ XGBoost ]
Iteration number: 57 56s (1s) (97s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6115800466988632
[ XGBoost ]
Iteration number: 58 56s (1s) (97s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6157424437586412
[ XGBoost ]
Iteration number: 59 57s (1s)

[ XGBoost ]
Iteration number: 7 9s (1s) (123s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.741643179725608
[ XGBoost ]
Iteration number: 8 10s (1s) (122s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8038583434604454
[ XGBoost ]
Iteration number: 9 10s (1s) (116s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8587569681815945
[ XGBoost ]
Iteration number: 10 11s (1s) (111s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8700942850108059
[ XGBoost ]
Iteration number: 11 12s (1s) (109s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0604155460665348
[ XGBoost ]
Iteration number: 12 13s (1s) (106s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.13267363028912
[ XGBoost ]
Iteration number: 13 13s (1s) (102s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0941616105388685
[ XGBoost ]
Iteration number: 14 15s (1s) (104s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1075110441090505
[ XGBoost ]
Iteration number: 15 16s (1

[ XGBoost ]
Iteration number: 75 86s (1s) (115s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9864352699024398
[ XGBoost ]
Iteration number: 76 87s (1s) (114s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9915157813963846
[ XGBoost ]
Iteration number: 77 88s (1s) (115s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.981239600007462
[ XGBoost ]
Iteration number: 78 89s (1s) (114s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9804794856090492
[ XGBoost ]
Iteration number: 79 90s (1s) (114s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9905861145540749
[ XGBoost ]
Iteration number: 80 90s (1s) (113s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9585399289855953
[ XGBoost ]
Iteration number: 81 91s (1s) (112s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9659030212770823
[ XGBoost ]
Iteration number: 82 93s (1s) (113s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9715610507826756
[ XGBoost ]
Iteration number: 83 

[ XGBoost ]
Iteration number: 31 20s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.035819649217224
[ XGBoost ]
Iteration number: 32 21s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0399988761725405
[ XGBoost ]
Iteration number: 33 21s (1s) (65s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0345634859139057
[ XGBoost ]
Iteration number: 34 25s (1s) (74s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0490207094037678
[ XGBoost ]
Iteration number: 35 26s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0742616354366596
[ XGBoost ]
Iteration number: 36 26s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0746087842665488
[ XGBoost ]
Iteration number: 37 27s (1s) (73s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0849465940746477
[ XGBoost ]
Iteration number: 38 30s (1s) (78s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0754198987464605
[ XGBoost ]
Iteration number: 39 31s (1s)

[ XGBoost ]
Iteration number: 100 84s (1s) (84s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2328298355193086

{'model_list': [<models.classifiers.XGboost object at 0x1a2510d6d0>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes': None, 'num_stages': 1, 'pipeline_stages': ['classifier'], 'name': '[ XGBoost ]', 'analysis_mode': None, 'analysis_type': None}
{'model_list': [<models.classifiers.XGboost object at 0x1a2510d790>], 'explained': '[ *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes': None, 'num_stages': 1, 'pipeline_stages': ['classifier'], 'name': '[ XGBoost ]', 'analysis_mode': None, 'analysis_type': None}
{'model_list': [<models.classifiers.XGboost object at 0x1a2510d890>], 'explained': '[ *GBoost is an open-source software libra

In [224]:
array1 = []
with open('../../../AutoPrognosisThings/outputs/agg_CardiacArrhythmias_with_prob.txt') as f:
    for i in range(10):
        array1.append([float(x) for x in next(f).split()])
array2 = []
with open('../../../AutoPrognosisThings/outputs/agg_CardiacArrhythmias.txt') as f:
    for i in range(10):
        array2.append([float(x) for x in next(f).split()])

result= stats.ttest_ind(array1, array2)
result

Ttest_indResult(statistic=array([-1.03971023]), pvalue=array([0.31223976]))

In [203]:
!rm ../../../AutoPrognosisThings/outputs/agg_coronaryArtery_with_prob.txt

mylist=[]

for count in range(10):
    !rm ../../../AutoPrognosisThings/outputs/result.json
    !python3 autoprognosis.py\
    -i ../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_coronaryArtery.csv\
    -iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
    --target outcome \
    -o ../../../AutoPrognosisThings/outputs \
    --it 100 \
    --cv 0 \
    --nstage 4 \
    --modelindexes 2\
    --num_components 1\
    --kernel_freq 100
    with open('../../../AutoPrognosisThings/outputs/result.json', 'r') as outfile:
        with open("../../../AutoPrognosisThings/outputs/agg_coronaryArtery_with_prob.txt", "a") as agg:
            json_object=json.load(outfile)
            result=max(json_object['clf']['roc_lst'][0], json_object['clf_ens']['roc_lst'][0])
            agg.write(str(result) + "\n")
            mylist.append(result)
            
seriesObj=pd.Series(mylist)
with open("../../../AutoPrognosisThings/outputs/agg_coronaryArtery_with_prob.txt", "a") as agg:
    agg.write('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")
print('mean: ' + str(seriesObj.mean()) + ' std: ' + str(seriesObj.std()) + "\n")

rm: ../../../AutoPrognosisThings/outputs/agg_coronaryArtery_with_prob.txt: No such file or directory
[ XGBoost ]
[ XGBoost ]
Iteration number: 1 1s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 1s (1s) (65s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0000000000000029
[ XGBoost ]
Iteration number: 3 2s (1s) (67s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7618453694212898
[ XGBoost ]
Iteration number: 4 3s (1s) (65s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7214000316827397
[ XGBoost ]
Iteration number: 5 3s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.944524106156648
[ XGBoost ]
Iteration number: 6 4s (1s) (63s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.125673485033473
[ XGBoost ]
Iteration number: 7 4s (1s) (64s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9878510428757171
[ XGBoost ]
Iteration number: 8 5s (1s) (63s), Current pipelines:  [[[ XGBoost 

[ XGBoost ]
Iteration number: 68 64s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7726090577707017
[ XGBoost ]
Iteration number: 69 65s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7670339546149774
[ XGBoost ]
Iteration number: 70 65s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7684829119247665
[ XGBoost ]
Iteration number: 71 66s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7647892998478623
[ XGBoost ]
Iteration number: 72 67s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7595487002132052
[ XGBoost ]
Iteration number: 73 67s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7693749934765004
[ XGBoost ]
Iteration number: 74 68s (1s) (93s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7645976264264346
[ XGBoost ]
Iteration number: 75 69s (1s) (92s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7644491608577025
[ XGBoost ]
Iteration number: 76 70s (1s

[ XGBoost ]
Iteration number: 25 36s (1s) (146s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7844409807193432
[ XGBoost ]
Iteration number: 26 37s (1s) (142s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8083316090083015
[ XGBoost ]
Iteration number: 27 38s (1s) (140s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7919881093052854
[ XGBoost ]
Iteration number: 28 39s (1s) (138s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7783343685768759
[ XGBoost ]
Iteration number: 29 40s (1s) (137s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7742498417361475
[ XGBoost ]
Iteration number: 30 41s (1s) (137s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7556119303150732
[ XGBoost ]
Iteration number: 31 42s (1s) (135s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7635893273031215
[ XGBoost ]
Iteration number: 32 43s (1s) (133s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.7676399331412331
[ XGBoost ]
Iteration number: 33

[ XGBoost ]
Iteration number: 93 101s (1s) (108s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6443717552260066
[ XGBoost ]
Iteration number: 94 102s (1s) (109s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.641659812919471
[ XGBoost ]
Iteration number: 95 103s (1s) (109s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.640531833850765
[ XGBoost ]
Iteration number: 96 104s (1s) (108s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6455392443423722
[ XGBoost ]
Iteration number: 97 106s (1s) (109s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6424121800382162
[ XGBoost ]
Iteration number: 98 107s (1s) (109s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6424947542880122
[ XGBoost ]
Iteration number: 99 109s (1s) (110s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6393479586518116
[ XGBoost ]
Iteration number: 100 110s (1s) (110s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6356222548371019

{'model_list': [<models.

[ XGBoost ]
Iteration number: 49 46s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6283196832965791
[ XGBoost ]
Iteration number: 50 47s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6285979051052164
[ XGBoost ]
Iteration number: 51 48s (1s) (95s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6263710127175285
[ XGBoost ]
Iteration number: 52 49s (1s) (95s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6242952890108007
[ XGBoost ]
Iteration number: 53 50s (1s) (95s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6220804561237667
[ XGBoost ]
Iteration number: 54 51s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6178687670115101
[ XGBoost ]
Iteration number: 55 52s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6121194313420334
[ XGBoost ]
Iteration number: 56 53s (1s) (94s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.6138751727533166
[ XGBoost ]
Iteration number: 57 53s (1s

...

[ XGBoost ]
Iteration number: 37 28s (1s) (76s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8304454171507336
[ XGBoost ]
Iteration number: 38 29s (1s) (76s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9194923314113216
[ XGBoost ]
Iteration number: 39 30s (1s) (76s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9134107189544137
[ XGBoost ]
Iteration number: 40 30s (1s) (76s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9057272626843054
[ XGBoost ]
Iteration number: 41 31s (1s) (76s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8958383795492326
[ XGBoost ]
Iteration number: 42 32s (1s) (76s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9061939882489585
[ XGBoost ]
Iteration number: 43 32s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9029292698663827
[ XGBoost ]
Iteration number: 44 33s (1s) (75s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9000124522244102
[ XGBoost ]
Iteration number: 45 34s (1s

[ XGBoost ]
[ XGBoost ]
Iteration number: 1 5s (5s) (518s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 6s (3s) (286s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 6s (2s) (211s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3919285865456634
[ XGBoost ]
Iteration number: 4 7s (2s) (176s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1778766621967818
[ XGBoost ]
Iteration number: 5 8s (2s) (154s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.294176915060618
[ XGBoost ]
Iteration number: 6 8s (1s) (140s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1290846736047158
[ XGBoost ]
Iteration number: 7 9s (1s) (128s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2855551222062724
[ XGBoost ]
Iteration number: 8 9s (1s) (118s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.272757857303095
[ XGBoost ]
Iteration number: 9 10s (1s) (111s), Current pipelines:  

Iteration number: 68 82s (1s) (121s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.927678308769259
[ XGBoost ]
Iteration number: 69 83s (1s) (120s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9298544539178126
[ XGBoost ]
Iteration number: 70 88s (1s) (125s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9312095194118184
[ XGBoost ]
Iteration number: 71 89s (1s) (125s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9451295733340727
[ XGBoost ]
Iteration number: 72 90s (1s) (124s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9536563523854079
[ XGBoost ]
Iteration number: 73 91s (1s) (124s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9482069154963093
[ XGBoost ]
Iteration number: 74 91s (1s) (123s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9532603584445997
[ XGBoost ]
Iteration number: 75 93s (1s) (124s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9630655404419165
[ XGBoost ]
Iteration number: 76 98s (1s) (12

[ XGBoost ]
Iteration number: 24 14s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0563957499509522
[ XGBoost ]
Iteration number: 25 15s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.951706115520658
[ XGBoost ]
Iteration number: 26 16s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9621445413723269
[ XGBoost ]
Iteration number: 27 16s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9812390107907871
[ XGBoost ]
Iteration number: 28 17s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9682991468209239
[ XGBoost ]
Iteration number: 29 17s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9773058207923401
[ XGBoost ]
Iteration number: 30 18s (1s) (60s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.967504028718383
[ XGBoost ]
Iteration number: 31 19s (1s) (61s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9299226472720965
[ XGBoost ]
Iteration number: 32 20s (1s) 

[ XGBoost ]
Iteration number: 93 65s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8907180258286328
[ XGBoost ]
Iteration number: 94 66s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8880173526940164
[ XGBoost ]
Iteration number: 95 67s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8874053008222322
[ XGBoost ]
Iteration number: 96 68s (1s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8905145266344071
[ XGBoost ]
Iteration number: 97 69s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8828410766405489
[ XGBoost ]
Iteration number: 98 69s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8817057730637786
[ XGBoost ]
Iteration number: 99 70s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8770568993461826
[ XGBoost ]
Iteration number: 100 71s (1s) (71s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.8780740687112646

{'model_list': [<models.classifiers.XG

In [226]:
array1 = []
with open('../../../AutoPrognosisThings/outputs/agg_coronaryArtery_with_prob.txt') as f:
    for i in range(10):
        array1.append([float(x) for x in next(f).split()])
array2 = []
with open('../../../AutoPrognosisThings/outputs/agg_CoronaryArtery.txt') as f:
    for i in range(10):
        array2.append([float(x) for x in next(f).split()])

result= stats.ttest_ind(array1, array2)
result

Ttest_indResult(statistic=array([1.830706]), pvalue=array([0.08375193]))

## Run for image data set with manual and auto imputation

In [ ]:
!python3 autoprognosis.py\
-i ../../../AutoPrognosisThings/cardio_data/withImage_cardio_data_17_feature_noNan.csv\
--target outcome \
-o ../../../AutoPrognosisThings/outputs \
--it 15 \
--cv 3 \
--nstage 4 \
--modelindexes 2\
--num_components 1\
--kernel_freq 100

In [ ]:
!python3 autoprognosis_report.py -i ../../../AutoPrognosisThings/outputs --verbose 1

In [ ]:
!python3 autoprognosis.py\
-i ../../../AutoPrognosisThings/cardio_data/withImage_cardio_data_17_feature_nan.csv\
--target outcome \
-o ../../../AutoPrognosisThings/outputs \
--it 100 \
--cv 3 \
--nstage 4 \
--modelindexes 2\
--num_components 1\
--kernel_freq 100

In [ ]:
!python3 autoprognosis_report.py -i ../../../AutoPrognosisThings/outputs --verbose 1

# Run the model for image data set

## Run the model for 17 features

In [ ]:
X_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/withImage/train_df_noNan")
X_.set_index('eid', inplace=True)
Y_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/withImage/train_df_noNan_outcome_green")
Y_.set_index('eid', inplace=True)
df_all= X_.join(Y_)
df_all.isnull().sum()

In [ ]:
df_all.to_csv('../../../AutoPrognosisThings/cardio_data/withImage_cardio_data_17_feature_noNan.csv')

In [ ]:
X_= df_all.drop(columns=['outcome'])
Y_= df_all[['outcome']]

In [ ]:
metric = 'aucprc'
acquisition_type = 'MPI' # default and prefered is LCB but this generates excessive warnings, MPI is a good compromise.
#I changed kernel_freq=100 and Gibbs_iter=100
model.nmax_model= 4
AP_mdl   = model.AutoPrognosis_Classifier(
    metric=metric, CV=3, num_iter=15, kernel_freq=100, ensemble=True,
    ensemble_size=3, Gibbs_iter=100, burn_in=50, num_components=3, 
    acquisition_type=acquisition_type, my_model_indexes=[2])

In [ ]:
AP_mdl.fit(X_, Y_)

## Run the model for 7 features

In [83]:
X_= pd.read_csv("../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_CardiacArrhythmias.csv")
X_.set_index('eid', inplace=True)
Y_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/withImage/train_df_noNan_outcome_CardiacArrhythmias")
Y_.set_index('eid', inplace=True)
df_all= X_.join(Y_)

In [84]:
df_all= df_all[['gender', 'age-0', 'average-sys-0', 'history-of-diabetes', 'hypertention-medication-0', 'smoker',
                'average-BMI-0', 'prob', 'outcome']]

In [85]:
df_all.to_csv('../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_CardiacArrhythmias.csv')

In [ ]:
X_= pd.read_csv("../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_vascular.csv")
X_.set_index('eid', inplace=True)
Y_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/withImage/train_df_noNan_outcome_Vascular")
Y_.set_index('eid', inplace=True)
df_all= X_.join(Y_)

In [78]:
df_all= df_all[['gender', 'age-0', 'average-sys-0', 'history-of-diabetes', 'hypertention-medication-0', 'smoker',
                'average-BMI-0', 'prob', 'outcome']]

In [79]:
df_all.to_csv('../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_vascular.csv')

In [87]:
X_= pd.read_csv("../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_cereb.csv")
X_.set_index('eid', inplace=True)
Y_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/withImage/train_df_noNan_outcome_CEREBROVASCULAR")
Y_.set_index('eid', inplace=True)
df_all= X_.join(Y_)

In [88]:
df_all= df_all[['gender', 'age-0', 'average-sys-0', 'history-of-diabetes', 'hypertention-medication-0', 'smoker',
                'average-BMI-0', 'prob', 'outcome']]

In [89]:
df_all.to_csv('../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_cereb.csv')

In [90]:
X_= pd.read_csv("../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_coronaryArtery.csv")
X_.set_index('eid', inplace=True)
Y_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/withImage/train_df_noNan_outcome_CoronaryArtery")
Y_.set_index('eid', inplace=True)
df_all= X_.join(Y_)

In [91]:
df_all= df_all[['gender', 'age-0', 'average-sys-0', 'history-of-diabetes', 'hypertention-medication-0', 'smoker',
                'average-BMI-0', 'prob', 'outcome']]

In [92]:
df_all.to_csv('../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_coronaryArtery.csv')

In [93]:
X_= pd.read_csv("../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_green.csv")
X_.set_index('eid', inplace=True)
Y_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/withImage/train_df_noNan_outcome_green")
Y_.set_index('eid', inplace=True)
df_all= X_.join(Y_)

In [94]:
df_all= df_all[['gender', 'age-0', 'average-sys-0', 'history-of-diabetes', 'hypertention-medication-0', 'smoker',
                'average-BMI-0', 'prob', 'outcome']]

In [95]:
df_all.to_csv('../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_green.csv')

In [96]:
X_= pd.read_csv("../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_orange.csv")
X_.set_index('eid', inplace=True)
Y_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/withImage/train_df_noNan_outcome_orange")
Y_.set_index('eid', inplace=True)
df_all= X_.join(Y_)

In [97]:
df_all= df_all[['gender', 'age-0', 'average-sys-0', 'history-of-diabetes', 'hypertention-medication-0', 'smoker',
                'average-BMI-0', 'prob', 'outcome']]

In [98]:
df_all.to_csv('../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_orange.csv')

In [ ]:
X_= df_all.drop(columns=['outcome'])
Y_= df_all[['outcome']]

In [ ]:
metric = 'aucprc'
acquisition_type = 'MPI' # default and prefered is LCB but this generates excessive warnings, MPI is a good compromise.
#I changed kernel_freq=100 and Gibbs_iter=100
model.nmax_model= 4
AP_mdl   = model.AutoPrognosis_Classifier(
    metric=metric, CV=3, num_iter=3, kernel_freq=100, ensemble=True,
    ensemble_size=3, Gibbs_iter=100, burn_in=50, num_components=3, 
    acquisition_type=acquisition_type, my_model_indexes=[0,1,2])

In [ ]:
AP_mdl.fit(X_, Y_)

# Run the model by short simple python code

In [ ]:
#X_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/ICD10 codes of the paper/train_df_noNan")
X_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/Green ICD10 codes/train_df_noNan")
X_.set_index('eid', inplace=True)
#Y_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/ICD10 codes of the paper/train_df_noNan_outcome")
Y_= pd.read_feather("../../../AutoPrognosisThings/cardio_data/Green ICD10 codes/train_df_noNan_outcome")
Y_.set_index('eid', inplace=True)
df_all= X_.join(Y_)

In [ ]:
# make a small random dataset
df_all=df_all.reindex(np.random.permutation(df_all.index))
df_all=df_all[:20000]
df_all.to_csv('../../../AutoPrognosisThings/cardio_data/small_cardio_data.csv')

In [ ]:
X_= df_all.drop(columns=['outcome'])
Y_= df_all[['outcome']]

In [ ]:
metric = 'aucprc'
acquisition_type = 'MPI' # default and prefered is LCB but this generates excessive warnings, MPI is a good compromise.
#I changed kernel_freq=100 and Gibbs_iter=100
model.nmax_model= 4
AP_mdl   = model.AutoPrognosis_Classifier(
    metric=metric, CV=3, num_iter=3, kernel_freq=100, ensemble=True,
    ensemble_size=3, Gibbs_iter=100, burn_in=50, num_components=3, 
    acquisition_type=acquisition_type, my_model_indexes=[0,1,2], is_nan=False)

In [ ]:
AP_mdl.fit(X_, Y_)

## Computing model predictions

##### ~~~First element in the output is the predictions of a single model, the second element is the prediction of the ensemble~~~

In [ ]:
AP_mdl.predict(X_)

## Compute performance via multi-fold cross-validation

In [ ]:
model.evaluate_ens(X_, Y_, AP_mdl, n_folds=5, visualize=True)

## Visualize data...

In [ ]:
AP_mdl.visualize_data(X_)

## Visualize the model...

In [ ]:
AP_mdl.APReport()